<a href="https://colab.research.google.com/github/seonghunYang/big_contest2020/blob/master/EDA12_1%EC%B0%A8_%EC%B5%9C%EC%A2%85_%EB%B3%80%EC%88%98%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
path = './gdrive/My Drive/빅콘테스트2020/데이터/2020빅콘테스트 데이터분석분야-퓨쳐스리그_스포츠투아이_제공데이터(.CSV)_시즌별, 시트별 구분/'

In [2]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
team_pitcher = pd.read_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/최근/1차_최종_팀투수.csv', encoding='utf-8-sig')
pitcher = pd.read_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/최근/1차_최종_개인투수.csv', encoding='utf-8-sig')
team_hitter = pd.read_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/최근/1차_최종_팀타자.csv', encoding='utf-8-sig')
hitter = pd.read_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/최근/1차_최종_개인타자.csv', encoding='utf-8-sig')
team_data = pd.read_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/최근/1차_최종_공통.csv', encoding='utf-8-sig')

In [4]:
data_list = [team_pitcher, pitcher, team_hitter, hitter,team_data]

In [5]:
for data in data_list:
  data['일자'] = pd.to_datetime(data['일자'],format='%Y-%m-%d', errors='raise')

In [30]:
## 저장 공간
team_pitcher.to_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/최근/1차_최종_팀투수.csv', encoding='utf-8-sig', index=False)
pitcher.to_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/최근/1차_최종_개인투수.csv', encoding='utf-8-sig', index=False)
team_data.to_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/최근/1차_최종_공통.csv', encoding='utf-8-sig', index=False)
team_hitter.to_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/최근/1차_최종_팀타자.csv', encoding='utf-8-sig', index=False)
hitter.to_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/최근/1차_최종_개인타자.csv', encoding='utf-8-sig', index=False)

In [ ]:
# 변화 확인하기
team_pitcher.columns

Index(['게임키', '일자', '팀코드', '상대팀코드', '더블헤더코드', '초말', '완투', '결과', '홀드', '이닝*3',
       '투구수', '타자', '타수', '안타', '2루타', '3루타', '홈런', '도루', '도루실패', '희타', '희비',
       '4구', '고4', '사구', '삼진', '병살타', '폭투', '보크', '실책', '실점', '자책점', '득점권WHIP',
       '2점차이하WHIP', '345번타자WHIP', '시즌', '평균자책점', 'RA/9', 'ERC', 'FIP', 'LOB',
       '홈런율', '삼진율', 'K9', 'BB9', 'HR9', 'KBB', 'whip', 'P/G', 'P/IP', 'P/PA',
       'PFR', 'CYP', 'F-E', 'R-E', '총합연봉', '평균연봉', 'OAVG', 'OOBP', '단타',
       'OSLG', 'OOPS', 'kwERA', 'K%', 'BB%'],
      dtype='object')

OOPS수정

In [ ]:
team_pitcher['OOPS'] = team_pitcher['OOBP'] + team_pitcher['OSLG']
pitcher['OOPS'] = pitcher['OOBP'] + pitcher['OSLG']

소수점 3자리로통일

In [ ]:
team_pitcher = team_pitcher.round(3)
pitcher = pitcher.round(3)
team_hitter = team_hitter.round(3)
hitter = hitter.round(3)

공통 데이터 구하기

In [ ]:
team_data.to_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/최근/1차_최종_공통.csv', encoding='utf-8-sig', index=False)

In [ ]:
team_code = ['NC', 'OB', 'WO', 'HT', "LG", 'SS', 'KT', 'LT', 'SK', "HH"]
def createDate(row):
  previous = team_pitcher[(team_pitcher['팀코드'] == row['팀코드']) & (team_pitcher['시즌'] == row['시즌']) & (team_pitcher['일자'] < row['일자']) ].tail(1)
  if len(previous) == 0:
    row['경기간간격'] = 0
    return row
  previous = previous.sum()
  a = (row['일자'] - previous['일자'])
  # print(a)
  row['경기간간격'] = a.days
  # print(row['일자'],  previous['일자'], row['경기간간격'])
  return row

In [ ]:
team_data = team_data.apply(createDate, axis=1)

In [ ]:
team_data['월'] =  team_data['일자'].dt.month

In [ ]:

def team_encoding(row, team_code):
  if row['팀코드'] == team_code:
    row[team_code] = 1
  else:
    row[team_code] = 0
  return row

def createHome(row):
  if row['초말'] == 'B':
    row['초말'] = 1
  else:
    row['초말'] = 0
  return row

def createOutIn(row):
  if row['이닝*3'] < 24:
    row['우취'] = 1
    row['연장'] = 0
  elif row['이닝*3'] > 27:
    row['우취'] = 0
    row['연장'] = 1
  else:
    row['우취'] = 0
    row['연장'] = 0
  return row


In [ ]:
team_code = ['NC', 'OB', 'WO', 'HT', "LG", 'SS', 'KT', 'LT', 'SK', "HH"]

for team in team_code:
  team_data = team_data.apply(team_encoding, axis=1, team_code=team)


team_data = team_data.apply(createHome, axis=1)
team_data = team_data.apply(createOutIn, axis=1)

In [ ]:
# 함수 모아놓은 곳
def gainCreate(row):
    row['득점'] = int(team_data[(row['게임키'] == team_data['게임키']) & (team_data['팀코드'] == row['상대팀코드'])]['실점'])
    return row

def createInfo(row):
    previous = team_data[(team_data['팀코드'] == row['팀코드']) & (team_data['일자'] < row['일자']) & (team_data['시즌'] == row['시즌'])].copy()
    if len(previous) == 0:
        row['시작전총득점'] = 0
        row['시작전총실점'] = 0
        row['시작전평균득점'] = 0
        row['시작전평균실점'] = 0
    else:
        gainSum = previous['득점'].sum()
        gainMean = gainSum / len(previous)
        lossSum = previous['실점'].sum()
        lossMean = lossSum / len(previous)
        row['시작전총득점'] = gainSum
        row['시작전총실점'] = lossSum
        row['시작전평균득점'] = gainMean
        row['시작전평균실점'] = lossMean
    return row

def winP(row):
    previous = team_data[(team_data['팀코드'] == row['팀코드']) & (team_data['일자'] < row['일자']) & (team_data['시즌'] == row['시즌'])].copy()
    if len(previous) == 0:
        row['시작전승수'] = 0
        row['시작전패배수'] = 0
        row['시작전승률'] = 0
    else:
        winSum = len(previous[previous['결과'] == "W"])
        lossSum = len(previous[previous['결과'] == "L"])
        winMean = winSum / len(previous)
        row['시작전승수'] = winSum
        row['시작전패배수'] = lossSum
        row['시작전승률'] = winMean
    return row

In [ ]:
team_data = team_data.apply(gainCreate, axis=1)
team_data = team_data.apply(createInfo, axis=1)
team_data = team_data.apply(winP, axis=1)


In [ ]:
team_data

,게임키,일자,팀코드,상대팀코드,더블헤더코드,초말,결과,시즌,경기간간격,월,NC,OB,WO,HT,LG,SS,KT,LT,SK,HH,이닝*3,실점,우취,연장,득점,시작전총득점,시작전총실점,시작전평균득점,시작전평균실점,시작전승수,시작전패배수,시작전승률
0,20160401HHLG0,2016-04-01,LG,HH,0.0,0,W,2016.0,0.0,4.0,0,0,0,0,1,0,0,0,0,0,36,4,0,1,5,0,0,0.000000,0.000000,0,0,0.000000
1,20160401HHLG0,2016-04-01,HH,LG,0.0,0,L,2016.0,0.0,4.0,0,0,0,0,0,0,0,0,0,1,34,5,0,1,4,0,0,0.000000,0.000000,0,0,0.000000
2,20160401HTNC0,2016-04-01,NC,HT,0.0,0,W,2016.0,0.0,4.0,1,0,0,0,0,0,0,0,0,0,27,4,0,0,5,0,0,0.000000,0.000000,0,0,0.000000
3,20160401HTNC0,2016-04-01,HT,NC,0.0,0,L,2016.0,0.0,4.0,0,0,0,1,0,0,0,0,0,0,24,5,0,0,4,0,0,0.000000,0.000000,0,0,0.000000
4,20160401KTSK0,2016-04-01,SK,KT,0.0,0,L,2016.0,0.0,4.0,0,0,0,0,0,0,0,0,1,0,27,8,0,0,4,0,0,0.000000,0.000000,0,0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6395,20200719LTSS0,2020-07-19,LT,SS,0.0,0,W,2020.0,1.0,7.0,0,0,0,0,0,0,0,1,0,0,27,1,0,0,2,305,320,5.000000,5.245902,29,32,0.475410
6396,20200719OBHT0,2020-07-19,HT,OB,0.0,0,L,2020.0,1.0,7.0,0,0,0,1,0,0,0,0,0,0,27,8,0,0,4,307,282,5.032787,4.622951,33,28,0.540984
6397,20200719OBHT0,2020-07-19,OB,HT,0.0,0,W,2020.0,1.0,7.0,0,1,0,0,0,0,0,0,0,0,27,4,0,0,8,387,348,6.142857,5.523810,37,26,0.587302
6398,20200719WOSK0,2020-07-19,SK,WO,0.0,0,W,2020.0,1.0,7.0,0,0,0,0,0,0,0,0,1,0,27,3,0,0,4,248,338,3.875000,5.281250,20,44,0.312500


In [ ]:
team_data['피타고리안기대승률'] = (team_data['시작전총득점'] ** 2) / ((team_data['시작전총득점'] ** 2) + (team_data['시작전총실점'] ** 2))

In [9]:
team_data = team_data.fillna(0)

In [11]:
team_data

,게임키,일자,팀코드,상대팀코드,더블헤더코드,초말,결과,시즌,경기간간격,월,NC,OB,WO,HT,LG,SS,KT,LT,SK,HH,이닝*3,실점,우취,연장,득점,시작전총득점,시작전총실점,시작전평균득점,시작전평균실점,시작전승수,시작전패배수,시작전승률,피타고리안기대승률
0,20160401HHLG0,2016-04-01,LG,HH,0.0,0,W,2016.0,0.0,4.0,0,0,0,0,1,0,0,0,0,0,36,4,0,1,5,0,0,0.000000,0.000000,0,0,0.000000,0.000000
1,20160401HHLG0,2016-04-01,HH,LG,0.0,0,L,2016.0,0.0,4.0,0,0,0,0,0,0,0,0,0,1,34,5,0,1,4,0,0,0.000000,0.000000,0,0,0.000000,0.000000
2,20160401HTNC0,2016-04-01,NC,HT,0.0,0,W,2016.0,0.0,4.0,1,0,0,0,0,0,0,0,0,0,27,4,0,0,5,0,0,0.000000,0.000000,0,0,0.000000,0.000000
3,20160401HTNC0,2016-04-01,HT,NC,0.0,0,L,2016.0,0.0,4.0,0,0,0,1,0,0,0,0,0,0,24,5,0,0,4,0,0,0.000000,0.000000,0,0,0.000000,0.000000
4,20160401KTSK0,2016-04-01,SK,KT,0.0,0,L,2016.0,0.0,4.0,0,0,0,0,0,0,0,0,1,0,27,8,0,0,4,0,0,0.000000,0.000000,0,0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6395,20200719LTSS0,2020-07-19,LT,SS,0.0,0,W,2020.0,1.0,7.0,0,0,0,0,0,0,0,1,0,0,27,1,0,0,2,305,320,5.000000,5.245902,29,32,0.475410,0.476014
6396,20200719OBHT0,2020-07-19,HT,OB,0.0,0,L,2020.0,1.0,7.0,0,0,0,1,0,0,0,0,0,0,27,8,0,0,4,307,282,5.032787,4.622951,33,28,0.540984,0.542368
6397,20200719OBHT0,2020-07-19,OB,HT,0.0,0,W,2020.0,1.0,7.0,0,1,0,0,0,0,0,0,0,0,27,4,0,0,8,387,348,6.142857,5.523810,37,26,0.587302,0.552912
6398,20200719WOSK0,2020-07-19,SK,WO,0.0,0,W,2020.0,1.0,7.0,0,0,0,0,0,0,0,0,1,0,27,3,0,0,4,248,338,3.875000,5.281250,20,44,0.312500,0.349956


상대전적별 데이터 구하기

In [13]:
vs_hitter = pd.read_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/EDA7-개인타자_상대별_세이버매트릭스_합본.csv', encoding='utf-8-sig')
vs_team_hitter = pd.read_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/EDA7-팀타자_상대별_세이버매트릭스_합본.csv', encoding='utf-8-sig')
vs_pitcher = pd.read_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/EDA8-상대전적_개안투수_세이버 매트릭스_합본.csv', encoding='utf-8-sig')
vs_team_pitcher = pd.read_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/EDA8-상대전적_팀투수_세이버 매트릭스_합본.csv', encoding='utf-8-sig')
# team_data = pd.read_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/최근/1차_최종_공통.csv', encoding='utf-8-sig')

In [27]:
vs_team_pitcher.to_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/최근/1차_최종_상대_팀투수.csv', encoding='utf-8-sig', index=False)

In [14]:
vs_team_hitter

,Unnamed: 0,게임키,일자,팀코드,상대팀코드,더블헤더코드,초말,타자,타수,타점,득점,안타,2루타,3루타,홈런,도루,도루실패,희타,희비,4구,고4,사구,삼진,병살타,실책,잔루,득점권타율,득점권타수,득점권안타,시즌,단타,상대출루율,상대장타력,상대ops,상대타율,상대ISO,상대BABIP,상대BB/K,상대K%,상대BB%,상대wOBA,상대wRAA,상대wRC,상대HR%,상대GPA
0,0,20160401HHLG0,2016-04-01,LG,HH,0,B,47,42,4,5,9,2,0,1,2,1,1,0,4,0,0,11,0,0,8,0.333333,12,4,2016,6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1,20160401HHLG0,2016-04-01,HH,LG,0,T,52,46,4,4,13,2,0,0,0,0,3,0,3,0,0,10,1,2,12,0.200000,15,3,2016,11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2,20160401HTNC0,2016-04-01,NC,HT,0,B,36,30,5,5,9,2,0,2,0,0,1,0,5,0,0,9,1,1,7,0.142857,7,1,2016,5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,3,20160401HTNC0,2016-04-01,HT,NC,0,T,38,34,3,4,8,3,0,1,0,0,0,0,3,0,1,10,1,0,7,0.100000,10,1,2016,4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,4,20160401KTSK0,2016-04-01,SK,KT,0,B,36,36,4,4,10,5,0,1,0,0,0,0,0,0,0,7,1,1,5,0.375000,8,3,2016,4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6395,635,20200719LTSS0,2020-07-19,LT,SS,0,T,31,29,2,2,6,1,0,1,0,0,1,0,1,0,0,7,3,0,2,0.000000,2,0,2020,4,0.299663,0.314607,0.614270,0.228464,0.086142,0.263636,0.565217,0.154362,0.087248,0.288293,-14.656294,8.018131,0.010067,0.213500
6396,636,20200719OBHT0,2020-07-19,HT,OB,0,B,38,32,4,4,8,0,1,0,0,0,0,0,6,0,0,5,1,0,7,0.571429,7,4,2020,7,0.340000,0.389513,0.729513,0.265918,0.123596,0.303167,0.681818,0.144737,0.095395,0.333920,-2.931033,20.276013,0.013158,0.250378
6397,637,20200719OBHT0,2020-07-19,OB,HT,0,T,41,37,8,8,11,3,0,1,2,1,0,0,2,1,2,4,0,0,6,0.363636,11,4,2020,7,0.363934,0.433213,0.797147,0.314079,0.119134,0.360870,0.382979,0.152104,0.058252,0.362672,4.736070,28.247471,0.012945,0.272074
6398,638,20200719WOSK0,2020-07-19,SK,WO,0,B,37,30,3,4,6,0,0,0,0,0,0,0,7,0,0,10,0,0,9,0.250000,8,2,2020,6,0.306931,0.365942,0.672873,0.246377,0.119565,0.287081,0.377049,0.200000,0.075410,0.307934,-9.804971,13.402075,0.026230,0.229604


In [15]:
def createRC(row):
    previous = vs_team_hitter[(vs_team_hitter['시즌'] == row['시즌']) & (vs_team_hitter['팀코드'] == row['팀코드']) & (vs_team_hitter['상대팀코드'] == row['상대팀코드']) & (vs_team_hitter['일자'] < row['일자'])]
    if len(previous) == 0:
        row['RC'] = 0
        return row
    previous_sum = previous.sum()
    upA = previous_sum['안타']+previous_sum['4구']-previous_sum['도루실패']+previous_sum['사구']-previous_sum['병살타']
    upB = (previous_sum['단타']*1.125)+(1.69*previous_sum['2루타'])+(3.02*previous_sum['3루타'])+(3.73*previous_sum['홈런'])+ 0.29*(previous_sum['4구']-previous_sum['고4']+previous_sum['사구'])
    down = previous_sum['타수']+previous_sum['4구']+previous_sum['사구']+previous_sum['희비']+previous_sum['희타']
    if down == 0:
        row['RC'] = 0
        return row
    row['RC'] = ((2.4*down+upA)*(3*down+upB)/(9*down)) - (0.9*down)
    
    return row

def createRC27(row):
    previous = vs_team_hitter[(vs_team_hitter['시즌'] == row['시즌']) & (vs_team_hitter['팀코드'] == row['팀코드']) & (vs_team_hitter['상대팀코드'] == row['상대팀코드']) & (vs_team_hitter['일자'] < row['일자'])]
    if len(previous) == 0:
        row['RC27'] = 0
        return row
    previous_sum = previous.sum()
    down = previous_sum['타수'] - previous_sum['안타']+previous_sum['병살타']+previous_sum['도루실패']+previous_sum['희타']+previous_sum['희비']
    if down == 0:
        row['RC27'] = 0
        return row
    row['RC27'] = (row['RC']/down)*27
    
    return row


def createSPD(row):
    previous = vs_team_hitter[(vs_team_hitter['시즌'] == row['시즌']) & (vs_team_hitter['팀코드'] == row['팀코드']) & (vs_team_hitter['상대팀코드'] == row['상대팀코드']) & (vs_team_hitter['일자'] < row['일자'])]
    if len(previous) == 0:
        row['SPD'] = 0
        return row
    previous_sum = previous.sum()
    try:
        f1 = ((previous_sum['도루']+3)/(previous_sum['도루']+previous_sum['도루실패']+7)-0.4) * 20
        f2 = ((previous_sum['도루']+previous_sum['도루실패'])/(previous_sum['단타']+previous_sum['4구']+previous_sum['사구']))**0.5/0.07
        f3 = previous_sum['3루타']/(previous_sum['타수']-previous_sum['홈런']-previous_sum['삼진'])/ 0.02*10
        f4 = ((previous_sum['득점']-previous_sum['홈런'])/(previous_sum['안타']+previous_sum['4구']-previous_sum['홈런']-previous_sum['사구'])-0.1)/0.04
        f5 = (0.055-previous_sum['병살타']/(previous_sum['타수']-previous_sum['홈런']-previous_sum['삼진']))/0.005
    except ZeroDivisionError:
        row['SPD'] = 0
        return row
    row['SPD'] = (f1+f2+f3+f4+f5) / 5
    return row

def createXR(row):
    previous = vs_team_hitter[(vs_team_hitter['시즌'] == row['시즌']) & (vs_team_hitter['팀코드'] == row['팀코드']) & (vs_team_hitter['상대팀코드'] == row['상대팀코드']) & (vs_team_hitter['일자'] < row['일자'])]
    if len(previous) == 0:
        row['XR'] = 0
        return row
    previous_sum = previous.sum()
    try:
        Y1 = 0.5*previous_sum['단타']+0.72*previous_sum['2루타']+1.04*previous_sum['3루타']+1.44*previous_sum['홈런']
        Y2 = (previous_sum['사구']+previous_sum['4구']-previous_sum['고4'])*0.34+0.25*previous_sum['고4']+0.18*previous_sum['도루']-0.32*previous_sum['도루실패']
        Y3 = -(previous_sum['타수']-previous_sum['안타']-previous_sum['삼진'])*0.09-0.098*previous_sum['삼진']-0.37*previous_sum['병살타']+0.37*previous_sum['희비']+0.04*previous_sum['희타']
    except ZeroDivisionError:
        row['XR'] = 0
        return row
    row['XR'] = Y1+Y2+Y3
    return row

In [16]:
try:
  vs_team_hitter = vs_team_hitter.apply(createRC, axis=1)
  vs_team_hitter = vs_team_hitter.apply(createRC27, axis=1)
  vs_team_hitter = vs_team_hitter.apply(createSPD, axis=1)
  vs_team_hitter = vs_team_hitter.apply(createXR, axis=1)
except:
  vs_team_hitter.to_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/최근/1차_최종_상대_팀타자.csv', encoding='utf-8-sig', index=False)
finally:
  vs_team_hitter.to_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/최근/1차_최종_상대_팀타자.csv', encoding='utf-8-sig', index=False)

In [17]:
vs_team_hitter

,Unnamed: 0,게임키,일자,팀코드,상대팀코드,더블헤더코드,초말,타자,타수,타점,득점,안타,2루타,3루타,홈런,도루,도루실패,희타,희비,4구,고4,사구,삼진,병살타,실책,잔루,득점권타율,득점권타수,득점권안타,시즌,단타,상대출루율,상대장타력,상대ops,상대타율,상대ISO,상대BABIP,상대BB/K,상대K%,상대BB%,상대wOBA,상대wRAA,상대wRC,상대HR%,상대GPA,RC,RC27,SPD,XR
0,0,20160401HHLG0,2016-04-01,LG,HH,0,B,47,42,4,5,9,2,0,1,2,1,1,0,4,0,0,11,0,0,8,0.333333,12,4,2016,6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000
1,1,20160401HHLG0,2016-04-01,HH,LG,0,T,52,46,4,4,13,2,0,0,0,0,3,0,3,0,0,10,1,2,12,0.200000,15,3,2016,11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000
2,2,20160401HTNC0,2016-04-01,NC,HT,0,B,36,30,5,5,9,2,0,2,0,0,1,0,5,0,0,9,1,1,7,0.142857,7,1,2016,5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000
3,3,20160401HTNC0,2016-04-01,HT,NC,0,T,38,34,3,4,8,3,0,1,0,0,0,0,3,0,1,10,1,0,7,0.100000,10,1,2016,4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000
4,4,20160401KTSK0,2016-04-01,SK,KT,0,B,36,36,4,4,10,5,0,1,0,0,0,0,0,0,0,7,1,1,5,0.375000,8,3,2016,4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6395,635,20200719LTSS0,2020-07-19,LT,SS,0,T,31,29,2,2,6,1,0,1,0,0,1,0,1,0,0,7,3,0,2,0.000000,2,0,2020,4,0.299663,0.314607,0.614270,0.228464,0.086142,0.263636,0.565217,0.154362,0.087248,0.288293,-14.656294,8.018131,0.010067,0.213500,25.754819,3.249440,4.448517,27.252
6396,636,20200719OBHT0,2020-07-19,HT,OB,0,B,38,32,4,4,8,0,1,0,0,0,0,0,6,0,0,5,1,0,7,0.571429,7,4,2020,7,0.340000,0.389513,0.729513,0.265918,0.123596,0.303167,0.681818,0.144737,0.095395,0.333920,-2.931033,20.276013,0.013158,0.250378,34.422495,4.404774,3.027312,34.698
6397,637,20200719OBHT0,2020-07-19,OB,HT,0,T,41,37,8,8,11,3,0,1,2,1,0,0,2,1,2,4,0,0,6,0.363636,11,4,2020,7,0.363934,0.433213,0.797147,0.314079,0.119134,0.360870,0.382979,0.152104,0.058252,0.362672,4.736070,28.247471,0.012945,0.272074,40.375889,5.166583,3.683077,40.444
6398,638,20200719WOSK0,2020-07-19,SK,WO,0,B,37,30,3,4,6,0,0,0,0,0,0,0,7,0,0,10,0,0,9,0.250000,8,2,2020,6,0.306931,0.365942,0.672873,0.246377,0.119565,0.287081,0.377049,0.200000,0.075410,0.307934,-9.804971,13.402075,0.026230,0.229604,30.931506,3.796139,3.918947,31.132


In [21]:
def createPFR(row):
    previous = vs_team_pitcher[(vs_team_pitcher['시즌'] == row['시즌']) & (vs_team_pitcher['팀코드'] == row['팀코드'])& (vs_team_pitcher['상대팀코드'] == row['상대팀코드']) & (vs_team_pitcher['일자'] < row['일자'])]
    if len(previous) == 0:
        row['PFR'] = 0
        return row
    previous_sum = previous.sum()
    try:
        pg = (previous_sum["삼진"]+previous_sum["4구"])/(previous_sum["이닝*3"]/3)
    except ZeroDivisionError:
        pg = 0
    row["PFR"] = pg
    return row
def createPG(row):
    previous = vs_team_pitcher[(vs_team_pitcher['시즌'] == row['시즌']) & (vs_team_pitcher['팀코드'] == row['팀코드'])& (vs_team_pitcher['상대팀코드'] == row['상대팀코드']) & (vs_team_pitcher['일자'] < row['일자'])]
    if len(previous) == 0:
        row['P/G'] = 0
        return row
    previous_sum = previous.sum()
    try:
        pg = previous_sum["투구수"]/previous.count()[0]
    except ZeroDivisionError:
        pg = 0
    row["P/G"] = pg
    return row

def createPIP(row):
    previous = vs_team_pitcher[(vs_team_pitcher['시즌'] == row['시즌']) & (vs_team_pitcher['팀코드'] == row['팀코드'])& (vs_team_pitcher['상대팀코드'] == row['상대팀코드']) & (vs_team_pitcher['일자'] < row['일자'])]
    if len(previous) == 0:
        row['P/IP'] = 0
        return row
    previous_sum = previous.sum()
    try:
        pg = previous_sum["투구수"]/(previous_sum["이닝*3"]/3)
    except ZeroDivisionError:
        pg = 0
    row["P/IP"] = pg
    return row
def createPPA(row):
    previous = vs_team_pitcher[(vs_team_pitcher['시즌'] == row['시즌']) & (vs_team_pitcher['팀코드'] == row['팀코드'])& (vs_team_pitcher['상대팀코드'] == row['상대팀코드']) & (vs_team_pitcher['일자'] < row['일자'])]
    if len(previous) == 0:
        row['P/PA'] = 0
        return row
    previous_sum = previous.sum()
    try:
        pg = previous_sum["투구수"]/previous_sum["타자"]
    except ZeroDivisionError:
        pg = 0
    row["P/PA"] = pg
    return row

def createCYP(row):
    previous = vs_team_pitcher[(vs_team_pitcher['시즌'] == row['시즌']) & (vs_team_pitcher['팀코드'] == row['팀코드'])& (vs_team_pitcher['상대팀코드'] == row['상대팀코드']) & (vs_team_pitcher['일자'] < row['일자'])]
    if len(previous) == 0:
        row['CYP'] = 0
        return row
    previous_sum = previous.sum()
    try:
        pg = (previous_sum["이닝*3"]/6-previous_sum["자책점"])+previous_sum["삼진"]/10+len(previous[previous['결과'] == 'W'])
    except ZeroDivisionError:
        pg = 0
    row["CYP"] = pg
    return row


In [22]:
try:
  vs_team_pitcher = vs_team_pitcher.apply(createPFR, axis=1)
  vs_team_pitcher = vs_team_pitcher.apply(createPG, axis=1)
  vs_team_pitcher = vs_team_pitcher.apply(createPIP, axis=1)
  vs_team_pitcher = vs_team_pitcher.apply(createPPA, axis=1)
  vs_team_pitcher = vs_team_pitcher.apply(createCYP, axis=1)
  vs_team_pitcher['상대F-E'] = vs_team_pitcher['상대FIP'] - vs_team_pitcher['상대평균자책점']
  vs_team_pitcher['상대R-E'] = vs_team_pitcher['상대RA/9'] - vs_team_pitcher['상대평균자책점']
except:
  vs_team_pitcher.to_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/최근/1차_최종_상대_팀투수.csv', encoding='utf-8-sig', index=False)
  print('오류발생')
finally:
  vs_team_pitcher.to_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/최근/1차_최종_상대_팀투수.csv', encoding='utf-8-sig', index=False)

오류발생


In [26]:
vs_team_pitcher['상대F-E'] = vs_team_pitcher['상대FIP'] - vs_team_pitcher['상대평균자책점']
vs_team_pitcher['상대R-E'] = vs_team_pitcher['상대RA/9'] - vs_team_pitcher['상대평균자책점']

F-E, R-E 수정

In [23]:
team_pitcher

,게임키,일자,팀코드,상대팀코드,더블헤더코드,초말,완투,결과,홀드,이닝*3,투구수,타자,타수,안타,2루타,3루타,홈런,도루,도루실패,희타,희비,4구,고4,사구,삼진,병살타,폭투,보크,실책,실점,자책점,득점권WHIP,2점차이하WHIP,345번타자WHIP,시즌,평균자책점,RA/9,ERC,FIP,LOB,홈런율,삼진율,K9,BB9,HR9,KBB,whip,P/G,P/IP,P/PA,PFR,CYP,F-E,R-E,총합연봉,평균연봉,OAVG,OOBP,단타,OSLG,OOPS,kwERA,K%,BB%,월
0,20160401HHLG0,2016-04-01,LG,HH,0,B,0,W,0,36,184,52,46,13,2,0,0,0,0,3,0,3,0,0,10,1,0,0,0,4,4,0.643,1.286,2.400,2016,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,925700,14025.758,0.000,0.000,11,0.000,0.000,0.000,0.000,0.000,4
1,20160401HHLG0,2016-04-01,HH,LG,0,T,0,L,0,34,196,47,42,9,2,0,1,2,1,1,0,4,0,0,11,0,1,0,0,5,4,1.500,1.000,0.750,2016,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1454500,19655.405,0.000,0.000,6,0.000,0.000,0.000,0.000,0.000,4
2,20160401HTNC0,2016-04-01,NC,HT,0,B,0,W,0,27,143,38,34,8,3,0,1,0,0,0,0,3,0,1,10,1,2,0,0,4,4,1.333,1.038,2.143,2016,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,872200,13017.910,0.000,0.000,4,0.000,0.000,0.000,0.000,0.000,4
3,20160401HTNC0,2016-04-01,HT,NC,0,T,0,L,0,24,147,36,30,9,2,0,2,0,0,1,0,5,0,0,9,1,0,0,0,5,5,0.500,1.696,1.875,2016,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1058700,15801.493,0.000,0.000,5,0.000,0.000,0.000,0.000,0.000,4
4,20160401KTSK0,2016-04-01,SK,KT,0,B,0,L,0,27,151,42,36,11,0,0,2,2,1,0,1,4,0,1,7,0,0,0,0,8,8,1.000,2.357,2.250,2016,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,955500,14051.471,0.000,0.000,9,0.000,0.000,0.000,0.000,0.000,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6395,20200719LTSS0,2020-07-19,LT,SS,0,T,0,W,2,27,155,34,31,7,0,0,1,2,1,0,0,3,0,0,9,1,0,0,0,1,1,0.600,1.111,1.500,2020,4.818,5.262,4.410,4.923,0.680,0.030,0.174,6.775,3.091,1.167,2.191,1.401,149.557,16.668,3.843,1.111,50.867,0.327,0.306,1067700,16682.812,0.271,0.335,6,0.423,0.758,4.308,0.174,0.083,7
6396,20200719OBHT0,2020-07-19,HT,OB,0,B,0,L,0,27,137,41,37,11,3,0,1,2,1,0,0,2,1,2,4,0,0,0,0,8,8,2.143,1.800,0.750,2020,4.292,4.747,3.970,4.190,0.685,0.021,0.199,7.676,3.114,0.791,2.465,1.367,145.836,16.638,3.881,1.203,90.933,-0.538,0.553,998090,15842.698,0.266,0.331,7,0.398,0.729,3.992,0.199,0.082,7
6397,20200719OBHT0,2020-07-19,OB,HT,0,T,0,W,1,27,177,38,32,8,0,1,0,0,0,0,0,6,0,0,5,1,0,0,0,4,4,3.000,1.000,1.875,2020,4.999,5.576,4.742,4.416,0.654,0.019,0.167,6.698,3.189,0.769,2.101,1.520,154.794,17.363,3.879,1.113,47.633,-0.947,0.614,975000,15476.190,0.289,0.352,7,0.421,0.773,4.393,0.166,0.082,7
6398,20200719WOSK0,2020-07-19,SK,WO,0,B,0,W,0,27,134,37,33,7,0,0,1,1,0,0,0,3,0,1,4,1,0,0,0,3,3,1.500,1.105,2.143,2020,4.874,5.419,4.805,4.960,0.684,0.023,0.176,7.103,4.249,0.930,1.672,1.530,154.641,17.631,3.934,1.268,40.967,-0.090,0.691,935800,15093.548,0.272,0.358,6,0.409,0.766,4.570,0.176,0.107,7


In [28]:
team_pitcher['F-E'] = team_pitcher['FIP'] - team_pitcher['평균자책점']
team_pitcher['R-E'] = team_pitcher['RA/9'] - team_pitcher['평균자책점']

In [29]:
team_pitcher

,게임키,일자,팀코드,상대팀코드,더블헤더코드,초말,완투,결과,홀드,이닝*3,투구수,타자,타수,안타,2루타,3루타,홈런,도루,도루실패,희타,희비,4구,고4,사구,삼진,병살타,폭투,보크,실책,실점,자책점,득점권WHIP,2점차이하WHIP,345번타자WHIP,시즌,평균자책점,RA/9,ERC,FIP,LOB,홈런율,삼진율,K9,BB9,HR9,KBB,whip,P/G,P/IP,P/PA,PFR,CYP,F-E,R-E,총합연봉,평균연봉,OAVG,OOBP,단타,OSLG,OOPS,kwERA,K%,BB%,월
0,20160401HHLG0,2016-04-01,LG,HH,0,B,0,W,0,36,184,52,46,13,2,0,0,0,0,3,0,3,0,0,10,1,0,0,0,4,4,0.643,1.286,2.400,2016,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,925700,14025.758,0.000,0.000,11,0.000,0.000,0.000,0.000,0.000,4
1,20160401HHLG0,2016-04-01,HH,LG,0,T,0,L,0,34,196,47,42,9,2,0,1,2,1,1,0,4,0,0,11,0,1,0,0,5,4,1.500,1.000,0.750,2016,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1454500,19655.405,0.000,0.000,6,0.000,0.000,0.000,0.000,0.000,4
2,20160401HTNC0,2016-04-01,NC,HT,0,B,0,W,0,27,143,38,34,8,3,0,1,0,0,0,0,3,0,1,10,1,2,0,0,4,4,1.333,1.038,2.143,2016,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,872200,13017.910,0.000,0.000,4,0.000,0.000,0.000,0.000,0.000,4
3,20160401HTNC0,2016-04-01,HT,NC,0,T,0,L,0,24,147,36,30,9,2,0,2,0,0,1,0,5,0,0,9,1,0,0,0,5,5,0.500,1.696,1.875,2016,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1058700,15801.493,0.000,0.000,5,0.000,0.000,0.000,0.000,0.000,4
4,20160401KTSK0,2016-04-01,SK,KT,0,B,0,L,0,27,151,42,36,11,0,0,2,2,1,0,1,4,0,1,7,0,0,0,0,8,8,1.000,2.357,2.250,2016,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,955500,14051.471,0.000,0.000,9,0.000,0.000,0.000,0.000,0.000,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6395,20200719LTSS0,2020-07-19,LT,SS,0,T,0,W,2,27,155,34,31,7,0,0,1,2,1,0,0,3,0,0,9,1,0,0,0,1,1,0.600,1.111,1.500,2020,4.818,5.262,4.410,4.923,0.680,0.030,0.174,6.775,3.091,1.167,2.191,1.401,149.557,16.668,3.843,1.111,50.867,0.105,0.444,1067700,16682.812,0.271,0.335,6,0.423,0.758,4.308,0.174,0.083,7
6396,20200719OBHT0,2020-07-19,HT,OB,0,B,0,L,0,27,137,41,37,11,3,0,1,2,1,0,0,2,1,2,4,0,0,0,0,8,8,2.143,1.800,0.750,2020,4.292,4.747,3.970,4.190,0.685,0.021,0.199,7.676,3.114,0.791,2.465,1.367,145.836,16.638,3.881,1.203,90.933,-0.102,0.455,998090,15842.698,0.266,0.331,7,0.398,0.729,3.992,0.199,0.082,7
6397,20200719OBHT0,2020-07-19,OB,HT,0,T,0,W,1,27,177,38,32,8,0,1,0,0,0,0,0,6,0,0,5,1,0,0,0,4,4,3.000,1.000,1.875,2020,4.999,5.576,4.742,4.416,0.654,0.019,0.167,6.698,3.189,0.769,2.101,1.520,154.794,17.363,3.879,1.113,47.633,-0.583,0.577,975000,15476.190,0.289,0.352,7,0.421,0.773,4.393,0.166,0.082,7
6398,20200719WOSK0,2020-07-19,SK,WO,0,B,0,W,0,27,134,37,33,7,0,0,1,1,0,0,0,3,0,1,4,1,0,0,0,3,3,1.500,1.105,2.143,2020,4.874,5.419,4.805,4.960,0.684,0.023,0.176,7.103,4.249,0.930,1.672,1.530,154.641,17.631,3.934,1.268,40.967,0.086,0.545,935800,15093.548,0.272,0.358,6,0.409,0.766,4.570,0.176,0.107,7
